# Evaluation example

This example uses the PromptTools repository with Sentence Transformers to evaluate the semantic similarity of generative outputs to an expected result.

Running this notebook requires a local `.gguf` model to be loaded on the device. This can be achieved by following the [llama.cpp](https://github.com/ggerganov/llama.cpp) repository instructions.

The particular models used here are [`Orca 2 13B`](https://huggingface.co/microsoft/Orca-2-13b) and [`Orca 2 7B`](https://huggingface.co/microsoft/Orca-2-7b) both with 4-bit quantisation.

## Import libraries

In [1]:
from prompttools.experiment import LlamaCppExperiment
import prompttools.utils as utils

## Define experiment paramaters

It is important to note that in the `LlamaCppExperiment` class we are passing `{"n_gpu_layers": [1]}` as the argument for `model_params` which enables GPU usage for Apple silicon.

This was successfully tested for the M1 MBP but has unknown effects for other devices and operating systems.

In [2]:
model_paths = [
    "../../models/Orca-2-13b-q4_0.gguf",
    "../../models/Orca-2-7b-q4_0.gguf",
]

prompts = [
    "Who was the first prime minister?",
    "Who was the first prime minister of the United Kingdom?",
]

temperatures = [0.0, 1.0]

call_params = dict(temperature=temperatures)

experiment = LlamaCppExperiment(
    model_paths, prompts, call_params=call_params, model_params={"n_gpu_layers": [1]}
)

## Run experiment

In [3]:
experiment.run()

llama_model_loader: loaded meta data with 22 key-value pairs and 363 tensors from ../../models/Orca-2-13b-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32003,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weight q4_0     [ 13824,  5120,     1,  

## Evaluate experiment

PromptTools provides utilities for evaluating generative outputs.

This example makes use of the `semantic_similarity` function and there are more availbale in the [PromptTools documentation](https://prompttools.readthedocs.io/en/latest/utils.html) such as `autoeval_binary_scoring` which uses GPT-4 as a "strong" judge.

In [4]:
experiment.evaluate(
    "similar_to_expected",
    utils.semantic_similarity,
    expected=["Robert Walpole"] * 8,
)

In [5]:
df = experiment.to_pandas_df(get_all_cols=False)

df.sort_values(by="similar_to_expected", ascending=False)

,model_path,temperature,prompt,client,response,latency,similar_to_expected
3,../../models/Orca-2-13b-q4_0.gguf,1.0,Who was the first prime minister of the United Kingdom?,<llama_cpp.llama.Llama object at 0x15b80ecd0>,"\nSir Robert Walpole was the first Prime Minister of the United Kingdom, although the title ""Prime Minister"" was not then in use. He held office from 1721 to 1742.",6.942941,0.469645
1,../../models/Orca-2-13b-q4_0.gguf,0.0,Who was the first prime minister of the United Kingdom?,<llama_cpp.llama.Llama object at 0x15b80ecd0>,"\nThe first Prime Minister of the United Kingdom was Sir Robert Walpole, who held office from 1721 to 1742.",4.844360,0.468164
5,../../models/Orca-2-7b-q4_0.gguf,0.0,Who was the first prime minister of the United Kingdom?,<llama_cpp.llama.Llama object at 0x15d0d9150>,"\nЋерё 18th century, the United Kingdom was formed by the union of England and Scotland. The first Prime Minister of the United Kingdom was Sir Robert Walpole, who served from 1\n922 to 1742. He is considered the first Prime Minister because he was the first person to hold the position that is equivalent to the modern-day Prime Minister in terms of power and influence. However, it's important to note that his role was not an official one at the time, but rather a de facto position.\n\nSir Robert Walpole was a member of the Wh",10.380895,0.446570
7,../../models/Orca-2-7b-q4_0.gguf,1.0,Who was the first prime minister of the United Kingdom?,<llama_cpp.llama.Llama object at 0x15d0d9150>,"\n Pythagoras is known as the father of geometry, but he lived in ancient Greece. He had nothing to do with the formation of the UK, which came into existence much later. The question seems to be asking about the first leader of the British government, rather than a prime minister. The first leader of the British government was actually Sir Robert Walpole, who served as the de facto head of the British Whig-Courtier administration from 1\n/4 1715",8.311724,0.344184
0,../../models/Orca-2-13b-q4_0.gguf,0.0,Who was the first prime minister?,<llama_cpp.llama.Llama object at 0x15b80ecd0>,"\nThe first Prime Minister of India was Jawaharlal Nehru. He served as the Prime Minister from 1947, when India gained independence, until his death in 1964.",16.490837,0.133516
4,../../models/Orca-2-7b-q4_0.gguf,0.0,Who was the first prime minister?,<llama_cpp.llama.Llama object at 0x15d0d9150>,"\n The first Prime Minister of India was Jawaharlal Nehru, who took office on 15th August 1947. He was a prominent leader in the Indian independence movement and served as the Prime Minister until his death in 1964.",10.785691,0.127394
6,../../models/Orca-2-7b-q4_0.gguf,1.0,Who was the first prime minister?,<llama_cpp.llama.Llama object at 0x15d0d9150>,"\n The first Prime Minister of India was Jawaharlal Nehru, who took office on August 15, 1947, the same day that India gained independence from British rule. He was the leader of the Indian National Congress and an influential figure in the Indian independence movement.",5.134083,0.124417
2,../../models/Orca-2-13b-q4_0.gguf,1.0,Who was the first prime minister?,<llama_cpp.llama.Llama object at 0x15b80ecd0>,"\nThe first Prime Minister of India was Jawaharlal Nehru. He took office on August 15, 1947, following India's independence from British rule.",6.207262,0.096734
